## 1. Setup and Imports

In [ ]:
import sys
sys.path.append('UrbanEV-main/code')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print("Libraries imported successfully!")

## 2. Define LSTM Model Architecture

In [ ]:
class Lstm(nn.Module):
    def __init__(self, seq, n_fea, node=275):
        super(Lstm, self).__init__()
        self.num_feat = n_fea
        self.nodes = node
        self.seq_len = seq
        self.lstm_hidden_dim = 16
        self.lstm = nn.LSTM(input_size=n_fea, hidden_size=self.lstm_hidden_dim, num_layers=2,
                            batch_first=True)
        self.linear = nn.Linear(seq * self.lstm_hidden_dim, 1)

    def forward(self, occ, extra_feat=None):  # occ.shape = [batch, node, seq]
        x = occ.unsqueeze(-1)
        if extra_feat is not None and extra_feat != 'None':
            x = torch.cat([occ.unsqueeze(-1), extra_feat], dim=-1)
        assert x.shape[-1] == self.num_feat, f"Number of features ({x.shape[-1]}) does not match n_fea ({self.num_feat})."

        bs = x.shape[0]
        x = x.view(bs * self.nodes, self.seq_len, self.num_feat)

        lstm_out, _ = self.lstm(x)  # lstm_out shape: [batch_size * node, seq_len, lstm_hidden_dim]
        lstm_out = lstm_out.reshape(bs, self.nodes, self.seq_len * self.lstm_hidden_dim)
        x = self.linear(lstm_out)
        x = torch.squeeze(x)
        return x

print("LSTM model defined")

## 3. Load Data and Configuration

In [ ]:
# Configuration parameters
class Args:
    def __init__(self):
        self.feat = 'occ'  # Feature type: occupancy
        self.pred_len = 3  # Prediction horizon (3 hours ahead)
        self.seq_len = 12  # Look-back window (12 hours)
        self.add_feat = 'None'  # No additional features
        self.pred_type = 'region'  # Predict all regions/stations
        self.fold = 6  # Cross-validation fold
        self.total_fold = 6  # Total number of folds (months)
        self.seed = 42
        self.bs = 32  # Batch size
        self.epoch = 50  # Training epochs

args = Args()

# Set random seed
torch.manual_seed(args.seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(args.seed)

# Train-validation-test split ratios
TRAIN_RATIO, VAL_RATIO, TEST_RATIO = 0.8, 0.1, 0.1

print(f"Configuration:")
print(f"  Feature: {args.feat}")
print(f"  Prediction horizon: {args.pred_len} hour(s)")
print(f"  Look-back window: {args.seq_len} hours")
print(f"  Prediction type: {args.pred_type}")
print(f"  Batch size: {args.bs}")
print(f"  Training epochs: {args.epoch}")
print(f"  Train/Val/Test ratio: {TRAIN_RATIO}/{VAL_RATIO}/{TEST_RATIO}")

In [ ]:
# Load station information
inf = pd.read_csv('UrbanEV-main/data/inf.csv', header=0, index_col=None)
print(f"Total stations: {len(inf)}")
print(f"\nStation information:")
print(inf.head())

In [ ]:
# Load occupancy data
occ_df = pd.read_csv('UrbanEV-main/data/occupancy.csv', header=0, index_col=0)
time = pd.to_datetime(occ_df.index)

# Normalize occupancy by number of charging piles
charge_count_dict = dict(zip(inf['TAZID'].astype(str), inf['charge_count']))
for col in occ_df.columns:
    charge_count = charge_count_dict[col]
    occ_df[col] = occ_df[col] / charge_count

feat = np.array(occ_df)

print(f"Data shape: {feat.shape}")
print(f"Date range: {time.min()} to {time.max()}")
print(f"Number of stations: {feat.shape[1]}")
print(f"Number of time steps: {feat.shape[0]}")

## 4. Create Dataset and DataLoaders

In [ ]:
# Split data based on fold
month_list = list(time.month.unique())
fold_time = time.month.isin(month_list[0:args.fold]).sum()

train_end = int(fold_time * TRAIN_RATIO)
valid_start = train_end
valid_end = int(valid_start + fold_time * VAL_RATIO)
test_start = valid_end
test_end = int(fold_time)

train_feat = feat[:train_end]
valid_feat = feat[valid_start:valid_end]
test_feat = feat[test_start:test_end]

print(f"Data split:")
print(f"  Training: {train_feat.shape} (indices 0 to {train_end})")
print(f"  Validation: {valid_feat.shape} (indices {valid_start} to {valid_end})")
print(f"  Test: {test_feat.shape} (indices {test_start} to {test_end})")
print(f"\nTime ranges:")
print(f"  Training: {time[0]} to {time[train_end-1]}")
print(f"  Validation: {time[valid_start]} to {time[valid_end-1]}")
print(f"  Test: {time[test_start]} to {time[test_end-1]}")

In [ ]:
# Define function to create sequences for RNN (matches original UrbanEV implementation)
def create_rnn_data(dataset, lookback, predict_time):
	"""
	Create sequences for RNN/LSTM training
	Args:
		dataset: numpy array of shape (timesteps, stations)
		lookback: length of input sequence
		predict_time: prediction horizon
	Returns:
		X: input sequences of shape (num_samples, seq_len, stations)
		y: target values of shape (num_samples, stations)
	"""
	x = []
	y = []
	for i in range(len(dataset) - lookback - predict_time):
		x.append(dataset[i:i + lookback])
		y.append(dataset[i + lookback + predict_time - 1])
	return np.array(x), np.array(y)

# Create sequences
train_occ, train_label = create_rnn_data(train_feat, args.seq_len, args.pred_len)
valid_occ, valid_label = create_rnn_data(valid_feat, args.seq_len, args.pred_len)
test_occ, test_label = create_rnn_data(test_feat, args.seq_len, args.pred_len)

print(f"Sequence data shapes:")
print(f"  Training: X={train_occ.shape}, y={train_label.shape}")
print(f"  Validation: X={valid_occ.shape}, y={valid_label.shape}")
print(f"  Test: X={test_occ.shape}, y={test_label.shape}")

In [ ]:
# Define Dataset class (matches original UrbanEV implementation)
class CreateDataset(Dataset):
	def __init__(self, occ, label, device):
		self.occ = torch.FloatTensor(occ)
		self.label = torch.FloatTensor(label)
		self.device = device
	
	def __len__(self):
		return len(self.occ)
	
	def __getitem__(self, idx):
		# occ shape: (batch, seq, node) -> transpose to (batch, node, seq)
		output_occ = torch.transpose(self.occ[idx, :, :], 0, 1).to(self.device)
		output_label = self.label[idx, :].to(self.device)
		return output_occ, output_label

# Create datasets and dataloaders
train_dataset = CreateDataset(train_occ, train_label, device)
train_loader = DataLoader(train_dataset, batch_size=args.bs, shuffle=True, drop_last=True)

valid_dataset = CreateDataset(valid_occ, valid_label, device)
valid_loader = DataLoader(valid_dataset, batch_size=len(valid_occ), shuffle=False)

test_dataset = CreateDataset(test_occ, test_label, device)
test_loader = DataLoader(test_dataset, batch_size=len(test_occ), shuffle=False)

print(f"DataLoaders created:")
print(f"  Training batches: {len(train_loader)}")
print(f"  Validation batches: {len(valid_loader)}")
print(f"  Test batches: {len(test_loader)}")

## 5. Initialize Model and Training Setup

In [ ]:
# Initialize model
n_fea = 1  # Only occupancy feature
num_nodes = feat.shape[1]

model = Lstm(seq=args.seq_len, n_fea=n_fea, node=num_nodes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.00001)
loss_func = nn.MSELoss()

print(f"Model initialized:")
print(f"  Number of stations: {num_nodes}")
print(f"  Sequence length: {args.seq_len}")
print(f"  Number of features: {n_fea}")
print(f"  LSTM hidden dimension: {model.lstm_hidden_dim}")
print(f"\nModel architecture:")
print(model)

## 6. Train Model

In [ ]:
from tqdm.notebook import tqdm

# Training loop
train_losses = []
valid_losses = []
best_valid_loss = float('inf')

print("Starting training...")
for epoch in tqdm(range(args.epoch), desc='Training'):
    # Training phase
    model.train()
    epoch_train_loss = 0
    for batch_idx, (occupancy, label) in enumerate(train_loader):
        optimizer.zero_grad()
        predict = model(occupancy, None)
        loss = loss_func(predict, label)
        loss.backward()
        optimizer.step()
        epoch_train_loss += loss.item()
    
    avg_train_loss = epoch_train_loss / len(train_loader)
    train_losses.append(avg_train_loss)
    
    # Validation phase
    model.eval()
    with torch.no_grad():
        for occupancy, label in valid_loader:
            predict = model(occupancy, None)
            valid_loss = loss_func(predict, label)
            valid_losses.append(valid_loss.item())
            
            # Save best model
            if valid_loss.item() < best_valid_loss:
                best_valid_loss = valid_loss.item()
                best_model_state = model.state_dict().copy()
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{args.epoch} - Train Loss: {avg_train_loss:.6f}, Valid Loss: {valid_loss.item():.6f}")

print(f"\nTraining completed!")
print(f"Best validation loss: {best_valid_loss:.6f}")

# Load best model
model.load_state_dict(best_model_state)

In [ ]:
# Plot training history
fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(train_losses, label='Training Loss', alpha=0.7, linewidth=2)
ax.plot(range(0, len(valid_losses) * len(train_loader), len(train_loader)), 
        valid_losses, label='Validation Loss', alpha=0.7, linewidth=2, marker='o')
ax.set_xlabel('Iteration')
ax.set_ylabel('Loss (MSE)')
ax.set_title('Training and Validation Loss')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 7. Test Model and Make Predictions

In [ ]:
# Make predictions on test set
model.eval()
predictions_list = []
labels_list = []

with torch.no_grad():
    for occupancy, label in test_loader:
        predict = model(occupancy, None)
        predictions_list.append(predict.cpu().numpy())
        labels_list.append(label.cpu().numpy())

predictions = np.concatenate(predictions_list, axis=0)
test_labels = np.concatenate(labels_list, axis=0)

print(f"Predictions shape: {predictions.shape}")
print(f"Test labels shape: {test_labels.shape}")
print(f"Predictions range: [{predictions.min():.4f}, {predictions.max():.4f}]")
print(f"Actual values range: [{test_labels.min():.4f}, {test_labels.max():.4f}]")

## 8. Evaluate Model Performance

In [ ]:
# Calculate metrics
def calculate_metrics(y_true, y_pred):
    """Calculate various regression metrics"""
    eps = 2e-2
    
    # Handle near-zero values for MAPE
    y_true_mape = y_true.copy()
    y_pred_mape = y_pred.copy()
    y_true_mape[np.where(y_true_mape <= eps)] = np.abs(y_true_mape[np.where(y_true_mape <= eps)]) + eps
    y_pred_mape[np.where(y_true_mape <= eps)] = np.abs(y_pred_mape[np.where(y_true_mape <= eps)]) + eps
    
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true_mape, y_pred_mape)
    rae = np.sum(np.abs(y_pred_mape - y_true_mape)) / np.sum(np.abs(np.mean(y_true_mape) - y_true_mape))
    
    return {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'MAPE': mape,
        'RAE': rae
    }

# Calculate overall metrics
metrics = calculate_metrics(test_labels, predictions)

print("=" * 60)
print("LSTM MODEL PERFORMANCE - ALL STATIONS")
print("=" * 60)
for metric_name, metric_value in metrics.items():
    print(f"{metric_name:10s}: {metric_value:.6f}")
print("=" * 60)

## 9. Visualize Results - Overall Statistics

In [ ]:
# Plot actual vs predicted scatter for all stations
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Scatter plot
axes[0].scatter(test_labels.flatten(), predictions.flatten(), 
               alpha=0.3, s=1, c='steelblue')
axes[0].plot([test_labels.min(), test_labels.max()], 
            [test_labels.min(), test_labels.max()], 
            'r--', linewidth=2, label='Perfect Prediction')
axes[0].set_xlabel('Actual Occupancy (Normalized)')
axes[0].set_ylabel('Predicted Occupancy (Normalized)')
axes[0].set_title('Actual vs Predicted - All Stations (LSTM)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Residuals histogram
residuals = test_labels.flatten() - predictions.flatten()
axes[1].hist(residuals, bins=50, edgecolor='black', alpha=0.7, color='coral')
axes[1].axvline(x=0, color='red', linestyle='--', linewidth=2)
axes[1].set_xlabel('Residuals (Actual - Predicted)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Residuals')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Mean residual: {residuals.mean():.6f}")
print(f"Std residual: {residuals.std():.6f}")

## 10. Visualize Results - Time Series for Selected Stations

In [ ]:
# Select a few representative stations to visualize
selected_stations = [0, 50, 100, 150, 200, 250]  # Indices of stations to plot
station_ids = occ_df.columns[selected_stations]

# Time indices for test data (accounting for sequence creation)
test_time = time[test_start + args.seq_len + args.pred_len - 1:test_end - 1]

fig, axes = plt.subplots(3, 2, figsize=(16, 12))
axes = axes.flatten()

for idx, (station_idx, station_id) in enumerate(zip(selected_stations, station_ids)):
    ax = axes[idx]
    
    # Plot actual vs predicted
    ax.plot(test_time[:len(test_labels)], test_labels[:, station_idx], 
           label='Actual', linewidth=1.5, alpha=0.8, color='steelblue')
    ax.plot(test_time[:len(predictions)], predictions[:, station_idx], 
           label='Predicted', linewidth=1.5, alpha=0.8, color='coral', linestyle='--')
    
    # Calculate station-specific metrics
    station_mae = mean_absolute_error(test_labels[:, station_idx], predictions[:, station_idx])
    
    ax.set_xlabel('Time')
    ax.set_ylabel('Occupancy (Normalized)')
    ax.set_title(f'Station {station_id} - MAE: {station_mae:.4f}')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 11. Per-Station Performance Analysis

In [ ]:
# Calculate metrics for each station
station_metrics = []

for station_idx in range(feat.shape[1]):
    station_id = occ_df.columns[station_idx]
    station_mae = mean_absolute_error(test_labels[:, station_idx], 
                                     predictions[:, station_idx])
    station_rmse = np.sqrt(mean_squared_error(test_labels[:, station_idx], 
                                              predictions[:, station_idx]))
    
    station_metrics.append({
        'Station_ID': station_id,
        'MAE': station_mae,
        'RMSE': station_rmse
    })

metrics_df = pd.DataFrame(station_metrics)

print("Per-Station Performance Summary:")
print(metrics_df.describe())
print(f"\nTop 10 Best Performing Stations (Lowest MAE):")
print(metrics_df.nsmallest(10, 'MAE'))
print(f"\nTop 10 Worst Performing Stations (Highest MAE):")
print(metrics_df.nlargest(10, 'MAE'))

In [ ]:
# Visualize distribution of station-level performance
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# MAE distribution
axes[0].hist(metrics_df['MAE'], bins=30, edgecolor='black', alpha=0.7, color='steelblue')
axes[0].axvline(x=metrics_df['MAE'].mean(), color='red', linestyle='--', 
               linewidth=2, label=f'Mean: {metrics_df["MAE"].mean():.4f}')
axes[0].set_xlabel('MAE')
axes[0].set_ylabel('Number of Stations')
axes[0].set_title('Distribution of MAE Across All Stations (LSTM)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# RMSE distribution
axes[1].hist(metrics_df['RMSE'], bins=30, edgecolor='black', alpha=0.7, color='coral')
axes[1].axvline(x=metrics_df['RMSE'].mean(), color='red', linestyle='--', 
               linewidth=2, label=f'Mean: {metrics_df["RMSE"].mean():.4f}')
axes[1].set_xlabel('RMSE')
axes[1].set_ylabel('Number of Stations')
axes[1].set_title('Distribution of RMSE Across All Stations (LSTM)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 12. Summary

In [ ]:
print("=" * 70)
print("LSTM MODEL - FINAL SUMMARY")
print("=" * 70)
print(f"\nModel Configuration:")
print(f"  Prediction Horizon: {args.pred_len} hour(s)")
print(f"  Look-back Window: {args.seq_len} hours")
print(f"  Number of Stations: {feat.shape[1]}")
print(f"  Test Set Size: {test_labels.shape[0]} time steps")
print(f"  LSTM Hidden Dim: {model.lstm_hidden_dim}")
print(f"  LSTM Layers: 2")
print(f"  Training Epochs: {args.epoch}")
print(f"  Batch Size: {args.bs}")

print(f"\nOverall Performance (All Stations):")
for metric_name, metric_value in metrics.items():
    print(f"  {metric_name:10s}: {metric_value:.6f}")

print(f"\nPer-Station Performance Statistics:")
print(f"  MAE - Mean: {metrics_df['MAE'].mean():.6f}, Std: {metrics_df['MAE'].std():.6f}")
print(f"  MAE - Min:  {metrics_df['MAE'].min():.6f}, Max: {metrics_df['MAE'].max():.6f}")
print(f"  RMSE - Mean: {metrics_df['RMSE'].mean():.6f}, Std: {metrics_df['RMSE'].std():.6f}")
print(f"  RMSE - Min:  {metrics_df['RMSE'].min():.6f}, Max: {metrics_df['RMSE'].max():.6f}")

print("\n" + "=" * 70)
print("The LSTM model uses recurrent neural networks to learn temporal patterns")
print("in the charging station occupancy data. It processes sequences of past")
print("observations to predict future values, capturing complex time dependencies.")
print("=" * 70)